In [1]:
import json
from tqdm.notebook import tqdm
from user_info import get_user_info
import random
from datetime import datetime
from dateutil.parser import parse

In [2]:
with open('../custom_data/llm_outputs/parsed_resumes_feb_march.json','r') as f:
    resp = json.load(f)

In [3]:
uid_list = list(resp.keys())

In [4]:
def parse_user_work_ex(info_json):
    work_ex = []
    try:
        for x in info_json['professional_info'][0]:
            
            user_dict = {}
            exp_dict = x
        
            keys = ['id','designation','fromExpMonth','fromExpYear','toExpMonth','toExpYear'] 
        
            for k in keys:
                user_dict[k] = exp_dict[k]
                
            user_dict['company'] = exp_dict['organization']['name']
            work_ex.append(user_dict)
    except:
        return work_ex
            
    return work_ex

In [5]:
check = {}
for uid in tqdm(uid_list):
    info_json = get_user_info(uid)
    work_ex = parse_user_work_ex(info_json)
    check[uid] = work_ex

  0%|          | 0/2408 [00:00<?, ?it/s]

In [8]:
uid = random.sample(uid_list,1)[0]

In [9]:
check[uid]

[{'id': 2500471,
  'designation': 'Director, HR',
  'fromExpMonth': '8',
  'fromExpYear': '2019',
  'toExpMonth': '0',
  'toExpYear': '0',
  'company': 'oyo'},
 {'id': 25214,
  'designation': 'Senior Manager HR',
  'fromExpMonth': '6',
  'fromExpYear': '2018',
  'toExpMonth': '8',
  'toExpYear': '2019',
  'company': 'Rivigo'},
 {'id': 419844,
  'designation': 'Chief Manager, Human Resources',
  'fromExpMonth': '1',
  'fromExpYear': '2013',
  'toExpMonth': '5',
  'toExpYear': '2018',
  'company': 'ICICI Bank'},
 {'id': 1715299,
  'designation': 'HR Associate',
  'fromExpMonth': '7',
  'fromExpYear': '2010',
  'toExpMonth': '1',
  'toExpYear': '2013',
  'company': 'State Bank of India'}]

In [10]:
resp[uid]

[{'company': 'OYO',
  'role': 'Program Manager',
  'start_date': '07/2020',
  'end_date': 'present'},
 {'company': 'Rivigo',
  'role': 'HR Partner',
  'start_date': '06/2018',
  'end_date': '08/2019'},
 {'company': 'ICICI Bank',
  'role': 'Chief Manager- HR',
  'start_date': '01/2013',
  'end_date': '06/2018'},
 {'company': 'State Bank of India',
  'role': 'HR Associate',
  'start_date': '07/2010',
  'end_date': '01/2013'}]

In [11]:
excess = []
for userid in tqdm(uid_list):
    if isinstance(resp[userid], list):
        try:
            if len(resp[userid])>len(check[userid]):
                excess.append(userid)
        except Exception as e:
            print(resp[userid])
            print(e)
            continue
    else:
        continue

  0%|          | 0/2408 [00:00<?, ?it/s]

In [12]:
len(excess)

1762

In [13]:
uid = random.sample(excess,1)[0]

In [14]:
check[uid]

[{'id': 2154615,
  'designation': 'Sr-Coordinator',
  'fromExpMonth': '12',
  'fromExpYear': '2007',
  'toExpMonth': '0',
  'toExpYear': '0',
  'company': 'Bain & Company'}]

In [15]:
resp[uid]

[{'company': 'Bain & Company',
  'role': 'Senior Coordinator - Global Client Advocacy',
  'start_date': '10/2017',
  'end_date': 'present'},
 {'company': 'Bain & Company',
  'role': 'Coordinator - Global Client Advocacy',
  'start_date': '06/2015',
  'end_date': '10/2017'},
 {'company': 'Bain & Company',
  'role': 'Associate Global CRM',
  'start_date': '04/2013',
  'end_date': '06/2015'},
 {'company': 'Bain & Company',
  'role': 'Junior CRM Associate- Global CRM Marketing',
  'start_date': '04/2011',
  'end_date': '04/2013'},
 {'company': 'Bain & Company',
  'role': 'Global Marketing Temp- Global CRM Marketing',
  'start_date': '01/2008',
  'end_date': '04/2011'},
 {'company': 'Bain & Company',
  'role': 'Junior CRM Associate- Global CRM Marketing',
  'start_date': '01/2006',
  'end_date': '12/2007'},
 {'company': 'Bharti Axa Life Insurance',
  'role': 'International Roaming Assistant',
  'start_date': '01/2004',
  'end_date': '12/2006'}]

In [17]:
uid_checks = {}
errors = []

for uid in tqdm(excess):
    es_jobs = []
    for idx, es_wex in enumerate(check[uid]):
        month = int(es_wex['fromExpMonth'])
        year = int(es_wex['fromExpYear'])
        start_time = datetime(year, month,1)
        es_jobs.append(start_time)

    try:
        comp = max(es_jobs)
    except:
        continue

    llm_exp = []
    for idx, llm_wex in enumerate(resp[uid]):
        try:
            start_time = parse(llm_wex['start_date'])
            if start_time>comp:
                if uid in uid_checks:
                    uid_checks[uid].append(llm_wex)
                else:
                    uid_checks[uid] = list()
                    uid_checks[uid].append(llm_wex)
        except Exception as e:
            errors.append(uid)
            pass

    

  0%|          | 0/1762 [00:00<?, ?it/s]

In [18]:
len(errors)

222